In [2]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import numpy as np
import re
import pickle
import pandas as pd
import re
import matplotlib.pyplot as plt

In [ ]:
#create dictionary for results, which will be used for further calculations
r_keys = ['L', 'W', 'L\xa0(1 OT)', 'L\xa0(2 OT)', 'W\xa0(2 OT)','W\xa0(1 OT)', 'W\xa0(3 OT)', 'L\xa0(3 OT)']
r_wins = [0,1,0,0,1,1,1,0] #1 for win
r_OT = [0,0,1,2,2,1,3,3] #overtimes played
r_MP = [40,40,45,50,50,45,55,55] #minutes played

results_dict = dict(zip(r_keys,zip(r_wins,r_OT,r_MP)))

# Data Cleaning and Feature Engineering

## Data Cleaning

In [3]:
team_lookup = pickle.load(open('teams_lookup.pickle', "rb" ))

In [76]:
combo_log = pickle.load(open('combo_log.pickle', "rb"))

In [84]:


combo_col_w_blank = ['url','Team','Date','Court','Opponent','Result','Tm','Opp','FG','FGA','FG%','3P','3PA','3P%','FT','FTA',
           'FT%','ORB','TRB','AST','STL','BLK','TOV','PF','blank','FG_O','FGA_O','FG%_O','3P_O','3PA_O','3P%_O','FT_O',
           'FTA_O','FT%_O','ORB_O','TRB_O','AST_O','STL_O','BLK_O','TOV_O','PF_O','url2','Team2','Date2','Court2','Opponent2','Result2','Tm2','Opp2','ORtg','DRtg','Pace','FTr','3PAr','TS%','TRB%',
           'AST%','STL%','BLK%','blank1','OeFG%','OTOV%','ORB%','OFT/FGA','blank2','DeFG%','DTOV%','DRB%','DFT/FGA']
num_cols = ['Tm','Opp','FG','FGA','FG%','3P','3PA','3P%','FT','FTA',
           'FT%','ORB','TRB','AST','STL','BLK','TOV','PF','FG_O','FGA_O','FG%_O','3P_O','3PA_O','3P%_O','FT_O',
           'FTA_O','FT%_O','ORB_O','TRB_O','AST_O','STL_O','BLK_O','TOV_O','PF_O','ORtg','DRtg','Pace','FTr','3PAr','TS%','TRB%',
           'AST%','STL%','BLK%','OeFG%','OTOV%','ORB%','OFT/FGA','DeFG%','DTOV%','DRB%','DFT/FGA']

In [109]:
def clean_gamelog(gamelog = combo_log, col_names_wb = combo_col_w_blank,
                  col_names = num_cols):
    '''
    Takes in gamelog and cleans the data
    input: gamelog as dataframe and stat_type. stat_type can be basic or advanced
    output: dataframe with clean gamelog
    '''
    games = pd.DataFrame.from_dict(gamelog, orient = "index")
    games.columns = col_names_wb
    games['Date'] = pd.to_datetime(games['Date'])
    games = games.drop(columns=['blank','url2','Team2','Date2','Court2','Opponent2','Result2','Tm2','Opp2','blank1','blank2'])
    for i in col_names:
        games[i] = pd.to_numeric(games[i])
    return games

In [125]:
#adding columns to help with further data manipulation
def helper_columns(df,team_lookup=team_lookup):
    '''
    this helps to clean data. update this here
    games = gamelog information by team ______
    columns = names of columns in the dataframe
    team_lookup = to get the right name
    
    output: dataframe cleaned!
    with columns we want
    
    '''
    hot_court = pd.get_dummies(df.Court)
    df['Home'],df['Away'] = hot_court[""],hot_court["@"]
    df['Wins'] = [results_dict[x][0] for x in df.Result]
    df['OT'] = [results_dict[x][1] for x in df.Result]
    df['MP']= [results_dict[x][2] for x in df.Result]
    return df

In [126]:
combo_log = pickle.load(open('combo_log.pickle', "rb"))
combo_log = clean_gamelog(combo_log)
combo_log = helper_columns(combo_log)

## Additional columns
spreads (see below)\
D1\
Win\
OT\
MP

In [127]:
#create lookup for D1 feature gen
#D1 = indicate if opponent is in Division 1 or not (from the data collection, only D1 teams included for gamelog but they sometimes play non-D1 schools)

team_df = pd.DataFrame.from_dict(team_lookup, orient = 'index')
D1 = dict(zip(team_df.loc[:,0],np.repeat(1,len(team_df.loc[:,0]))))

#removing all non-D1 games
combo_log['D1'] = [D1.get(x) or 0 for x in combo_log['Opponent']]
combo_log = combo_log[combo_log.D1 == 1]

In [128]:
#calculate various spreads

def calc_spreads(df):
    df['P_S'] = df.eval('Tm - Opp')
    df['FG_S'] = df.eval('FG - FG_O')
    df['FGA_S'] = df.eval('FGA - FGA_O')
    df['FG%_S'] = df['FG%'] - df['FG%_O']
    df['3P_S'] = df['3P'] - df['3P_O']
    df['3PA_S'] = df['3PA'] - df['3PA_O']
    df['3P%_S'] = df['3P%'] - df['3P%_O']
    df['FT_S'] = df.eval('FT - FT_O')
    df['FTA_S'] = df.eval('FTA - FTA_O')
    df['FT%_S'] = df['FT%'] - df['FT%_O']
    df['TRB_S'] = df.eval('TRB- TRB_O')
    df['AST_S'] = df.eval('AST - AST_O')
    df['STL_S'] = df.eval('STL - STL_O')
    df['BLK_S'] = df.eval('BLK - BLK_O')
    df['TOV_S'] = df.eval('TOV - TOV_O')
    df['PF_S'] = df.eval('PF - PF_O')
    df['FTr_S'] = df.eval('FTr - FT_O/FGA_O') 
    df['3PAr_S'] = df['3PA']/df['FG'] - df['3PA_O']/df['FG_O']
    df['TS%_S'] = df['TS%'] - df.eval('Opp/(2*(FGA_O+0.44*FTA_O))')
    df['TRB%_S'] = 2*df['TRB%'] - 1
    df['AST%_S'] = df.eval('(AST - AST_O)/Pace')
    df['STL%_S'] = df.eval('(STL - STL_O)/Pace')
    df['BLK%_S'] = df.eval('(BLK - BLK_O)/Pace')
    return df

In [129]:
combo_log = calc_spreads(combo_log)

In [133]:
with open('clean_combo_19.pickle', 'wb') as handle:
    pickle.dump(combo_log, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [132]:
len(combo_log)

11206

In [130]:
combo_log.columns

Index(['url', 'Team', 'Date', 'Court', 'Opponent', 'Result', 'Tm', 'Opp', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG_O', 'FGA_O', 'FG%_O', '3P_O',
       '3PA_O', '3P%_O', 'FT_O', 'FTA_O', 'FT%_O', 'ORB_O', 'TRB_O', 'AST_O',
       'STL_O', 'BLK_O', 'TOV_O', 'PF_O', 'ORtg', 'DRtg', 'Pace', 'FTr',
       '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'OeFG%', 'OTOV%', 'ORB%',
       'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'Home', 'Away', 'Wins',
       'OT', 'MP', 'D1', 'P_S', 'FG_S', 'FGA_S', 'FG%_S', '3P_S', '3PA_S',
       '3P%_S', 'FT_S', 'FTA_S', 'FT%_S', 'TRB_S', 'AST_S', 'STL_S', 'BLK_S',
       'TOV_S', 'PF_S', 'FTr_S', '3PAr_S', 'TS%_S', 'TRB%_S', 'AST%_S',
       'STL%_S', 'BLK%_S'],
      dtype='object')

## Preparing clean data file

In [136]:
def agg_df(games):
    '''
    This will compute the running stats for teams, with season-to-date info prior to each game
    '''
    df = pd.DataFrame(games['url'])
    df['Team'] = [team_lookup[x][0] for x in games['Team']]
    df['Date'] = games['Date']
    df['Opponent'] = games['Opponent']
    df['Spread'] = games['P_S']
    df['Win_Loss'] = games['Wins']
    df['Home'] = games['Home']
    df['Away'] = games['Away']
    
    # GP = Games Played prior to this game
    df['GP'] = games.groupby('Team')['Team'].transform(lambda x: x.expanding().count()-1)
    # get prior average stats for each team before the game
    for i in ['Wins','Tm', 'Opp', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'ORtg', 'DRtg', 'Pace', 'FTr',
       '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'OeFG%', 'OTOV%', 'ORB%',
       'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA','P_S', 'FG_S', 'FGA_S', 'FG%_S', '3P_S', '3PA_S',
       '3P%_S', 'FT_S', 'FTA_S', 'FT%_S', 'TRB_S', 'AST_S', 'STL_S', 'BLK_S','TOV_S', 'PF_S', 'FTr_S', '3PAr_S', 'TS%_S', 'TRB%_S', 'AST%_S',
       'STL%_S', 'BLK%_S']:
        df[i] = games.groupby('Team')[i].transform(lambda x: x.expanding().sum()-x)/df['GP']
    return df

In [137]:
df1 = agg_df(combo_log)

In [149]:
df1[0:40]

,url,Team,Date,Opponent,Spread,Win_Loss,Home,Away,GP,Wins,...,BLK_S,TOV_S,PF_S,FTr_S,3PAr_S,TS%_S,TRB%_S,AST%_S,STL%_S,BLK%_S
/cbb/boxscores/2018-11-09-23-california.htmlcalifornia,/cbb/boxscores/2018-11-09-23-california.html,California,2018-11-09,Yale,-17,0,0,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/cbb/boxscores/2018-11-13-22-california.htmlcalifornia,/cbb/boxscores/2018-11-13-22-california.html,California,2018-11-13,Hampton,14,1,1,0,1.0,0.000000,...,-3.000000,-3.000000,-12.000000,0.300077,-0.284444,-0.157175,86.600000,-0.105263,-0.013158,-0.039474
/cbb/boxscores/2018-11-19-19-california.htmlcalifornia,/cbb/boxscores/2018-11-19-19-california.html,California,2018-11-19,St. John's (NY),-3,0,0,0,2.0,0.500000,...,3.000000,-1.000000,-7.500000,0.275736,-0.350264,0.025401,90.600000,0.021592,0.000169,0.040992
/cbb/boxscores/2018-11-20-17-temple.htmlcalifornia,/cbb/boxscores/2018-11-20-17-temple.html,California,2018-11-20,Temple,-17,0,0,0,3.0,0.333333,...,2.000000,-0.333333,-4.333333,0.187861,-0.267992,0.018871,91.333333,-0.010370,0.000112,0.027328
/cbb/boxscores/2018-11-26-21-california.htmlcalifornia,/cbb/boxscores/2018-11-26-21-california.html,California,2018-11-26,Santa Clara,12,1,1,0,4.0,0.250000,...,0.500000,0.500000,-4.000000,0.214716,-0.191778,-0.007820,91.350000,-0.059476,-0.022072,0.005725
/cbb/boxscores/2018-12-01-22-saint-marys-ca.htmlcalifornia,/cbb/boxscores/2018-12-01-22-saint-marys-ca.html,California,2018-12-01,Saint Mary's,-13,0,0,1,5.0,0.400000,...,0.000000,-2.200000,-2.400000,0.182340,-0.252482,0.001186,89.360000,-0.047581,0.013241,-0.001038
/cbb/boxscores/2018-12-05-23-california.htmlcalifornia,/cbb/boxscores/2018-12-05-23-california.html,California,2018-12-05,San Francisco,-19,0,1,0,6.0,0.333333,...,-0.333333,-2.500000,-2.166667,0.149783,-0.253136,-0.031806,88.966667,-0.073825,0.008405,-0.006123
/cbb/boxscores/2018-12-08-22-california.htmlcalifornia,/cbb/boxscores/2018-12-08-22-california.html,California,2018-12-08,San Diego State,6,1,1,0,7.0,0.285714,...,0.142857,-1.857143,-2.142857,0.163037,-0.177047,-0.039429,87.971429,-0.072138,0.002775,0.001397
/cbb/boxscores/2018-12-15-22-california.htmlcalifornia,/cbb/boxscores/2018-12-15-22-california.html,California,2018-12-15,Cal Poly,1,1,1,0,8.0,0.375000,...,0.250000,-2.000000,-2.000000,0.157497,-0.124743,-0.035289,89.600000,-0.063121,0.000438,0.003212
/cbb/boxscores/2018-12-19-22-fresno-state.htmlcalifornia,/cbb/boxscores/2018-12-19-22-fresno-state.html,California,2018-12-19,Fresno State,-22,0,0,1,9.0,0.444444,...,0.333333,-2.333333,-1.666667,0.169912,-0.157300,-0.031272,90.266667,-0.061291,0.009029,0.004583


## Append (vs) Opponent Stats 

In [167]:
def vs_df(df):
    '''
    This will compute the running stats for teams, with season-to-date info prior to each game
    '''
    # GP = Games Played prior to this game
    winners = df[df.Win_Loss == 1]
    losers = df[df.Win_Loss == 0]
    
    winner_merge = pd.merge(winners, losers, left_on='url',right_on='url',how='outer',suffixes=('','_vs'))
    loser_merge = pd.merge(losers, winners, left_on='url',right_on='url',how='outer',suffixes=('','_vs'))
    final = pd.concat([winner_merge,loser_merge])
    return final

In [168]:
df_final = vs_df(df1)

In [174]:
df_final

,url,Team,Date,Opponent,Spread,Win_Loss,Home,Away,GP,Wins,...,BLK_S_vs,TOV_S_vs,PF_S_vs,FTr_S_vs,3PAr_S_vs,TS%_S_vs,TRB%_S_vs,AST%_S_vs,STL%_S_vs,BLK%_S_vs
0,/cbb/boxscores/2018-11-13-22-california.html,California,2018-11-13,Hampton,14,1,1,0,1.0,0.000000,...,-1.000000,1.000000,1.000000,0.053667,0.074879,-0.122053,104.200000,-0.087940,-0.075377,-0.012563
1,/cbb/boxscores/2018-11-26-21-california.html,California,2018-11-26,Santa Clara,12,1,1,0,4.0,0.250000,...,1.000000,1.800000,5.000000,-0.027312,0.323610,-0.041497,91.680000,-0.019679,-0.006209,0.015796
2,/cbb/boxscores/2018-12-08-22-california.html,California,2018-12-08,San Diego State,6,1,1,0,7.0,0.285714,...,0.750000,0.500000,-2.625000,0.200167,-0.210165,0.055071,101.275000,0.027414,-0.015284,0.010068
3,/cbb/boxscores/2018-12-15-22-california.html,California,2018-12-15,Cal Poly,1,1,1,0,8.0,0.375000,...,0.000000,1.714286,-0.428571,0.035119,0.094773,-0.045731,91.314286,-0.069181,-0.008425,0.000097
4,/cbb/boxscores/2018-12-21-22-california.html,California,2018-12-21,San Jose State,8,1,1,0,10.0,0.400000,...,0.111111,3.111111,-2.666667,0.283569,-0.113571,-0.025478,102.977778,-0.019670,-0.062934,0.000974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5598,/cbb/boxscores/2019-01-26-19-cleveland-state.html,Youngstown State,2019-01-26,Cleveland State,-10,0,0,1,19.0,0.210526,...,-0.263158,2.157895,-0.684211,0.079369,0.153413,-0.029673,92.736842,-0.032423,-0.034945,-0.002696
5599,/cbb/boxscores/2019-02-21-19-northern-kentucky...,Youngstown State,2019-02-21,Northern Kentucky,-7,0,0,1,26.0,0.384615,...,0.240000,-0.520000,1.200000,0.102166,0.031735,0.043565,105.000000,0.077575,0.003338,0.003380
5600,/cbb/boxscores/2019-02-23-15-wright-state.html,Youngstown State,2019-02-23,Wright State,-28,0,0,1,27.0,0.370370,...,-1.115385,-1.846154,-2.961538,0.167148,0.050100,-0.014979,101.761538,0.037687,0.008731,-0.016557
5601,/cbb/boxscores/2019-03-02-18-youngstown-state....,Youngstown State,2019-03-02,Cleveland State,-9,0,1,0,28.0,0.357143,...,-0.214286,1.250000,-0.535714,0.059117,0.146171,-0.019909,91.521429,-0.019571,-0.019866,-0.002160


In [180]:
pd.set_option('display.max_columns', None)
df_final[df_final['url'] == '/cbb/boxscores/2019-03-07-23-stanford.html']

,url,Team,Date,Opponent,Spread,Win_Loss,Home,Away,GP,Wins,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,OeFG%,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,P_S,FG_S,FGA_S,FG%_S,3P_S,3PA_S,3P%_S,FT_S,FTA_S,FT%_S,TRB_S,AST_S,STL_S,BLK_S,TOV_S,PF_S,FTr_S,3PAr_S,TS%_S,TRB%_S,AST%_S,STL%_S,BLK%_S,Team_vs,Date_vs,Opponent_vs,Spread_vs,Win_Loss_vs,Home_vs,Away_vs,GP_vs,Wins_vs,Tm_vs,Opp_vs,FG_vs,FGA_vs,FG%_vs,3P_vs,3PA_vs,3P%_vs,FT_vs,FTA_vs,FT%_vs,ORB_vs,TRB_vs,AST_vs,STL_vs,BLK_vs,TOV_vs,PF_vs,ORtg_vs,DRtg_vs,Pace_vs,FTr_vs,3PAr_vs,TS%_vs,TRB%_vs,AST%_vs,STL%_vs,BLK%_vs,OeFG%_vs,OTOV%_vs,ORB%_vs,OFT/FGA_vs,DeFG%_vs,DTOV%_vs,DRB%_vs,DFT/FGA_vs,P_S_vs,FG_S_vs,FGA_S_vs,FG%_S_vs,3P_S_vs,3PA_S_vs,3P%_S_vs,FT_S_vs,FTA_S_vs,FT%_S_vs,TRB_S_vs,AST_S_vs,STL_S_vs,BLK_S_vs,TOV_S_vs,PF_S_vs,FTr_S_vs,3PAr_S_vs,TS%_S_vs,TRB%_S_vs,AST%_S_vs,STL%_S_vs,BLK%_S_vs
7,/cbb/boxscores/2019-03-07-23-stanford.html,California,2019-03-07,Stanford,5,1,0,1,29.0,0.241379,69.172414,78.344828,24.137931,56.206897,0.430517,6.586207,19.034483,0.343586,14.310345,19.586207,0.726517,6.862069,26.586207,11.034483,7.172414,2.965517,10.965517,18.655172,99.006897,112.075862,69.862069,0.352862,0.338793,0.529759,44.989655,45.396552,10.048276,9.041379,0.489586,14.262069,21.148276,0.257828,0.580552,17.065517,72.824138,0.270000,-9.172414,-3.206897,1.103448,-0.068138,-2.379310,-3.931034,-0.039897,-0.379310,-0.793103,0.01769,-5.931034,-4.206897,2.241379,-0.206897,-2.448276,0.862069,0.082791,-0.041090,-0.085037,88.979310,-0.060665,0.030825,-0.002913,Stanford,2019-03-07,California,-5,0,1,0,29.0,0.517241,73.241379,71.862069,26.137931,56.655172,0.461759,6.758621,20.689655,0.317138,14.206897,21.000000,0.679379,8.206897,33.241379,12.793103,5.965517,4.551724,14.793103,18.793103,99.482759,98.275862,73.096552,0.376621,0.365862,0.550000,51.865517,48.848276,8.086207,10.148276,0.521069,18.327586,26.231034,0.255931,0.481897,14.631034,74.451724,0.223207,1.379310,-0.275862,-4.275862,0.026517,1.137931,4.103448,-0.019552,0.793103,1.517241,-0.00769,2.103448,2.000000,-0.448276,1.413793,2.724138,-0.724138,0.153427,0.188796,0.031462,102.731034,0.025630,-0.007512,0.018760
37,/cbb/boxscores/2019-03-07-23-stanford.html,Stanford,2019-03-07,California,-5,0,1,0,29.0,0.517241,73.241379,71.862069,26.137931,56.655172,0.461759,6.758621,20.689655,0.317138,14.206897,21.000000,0.679379,8.206897,33.241379,12.793103,5.965517,4.551724,14.793103,18.793103,99.482759,98.275862,73.096552,0.376621,0.365862,0.550000,51.865517,48.848276,8.086207,10.148276,0.521069,18.327586,26.231034,0.255931,0.481897,14.631034,74.451724,0.223207,1.379310,-0.275862,-4.275862,0.026517,1.137931,4.103448,-0.019552,0.793103,1.517241,-0.00769,2.103448,2.000000,-0.448276,1.413793,2.724138,-0.724138,0.153427,0.188796,0.031462,102.731034,0.025630,-0.007512,0.018760,California,2019-03-07,Stanford,5,1,0,1,29.0,0.241379,69.172414,78.344828,24.137931,56.206897,0.430517,6.586207,19.034483,0.343586,14.310345,19.586207,0.726517,6.862069,26.586207,11.034483,7.172414,2.965517,10.965517,18.655172,99.006897,112.075862,69.862069,0.352862,0.338793,0.529759,44.989655,45.396552,10.048276,9.041379,0.489586,14.262069,21.148276,0.257828,0.580552,17.065517,72.824138,0.270000,-9.172414,-3.206897,1.103448,-0.068138,-2.379310,-3.931034,-0.039897,-0.379310,-0.793103,0.01769,-5.931034,-4.206897,2.241379,-0.206897,-2.448276,0.862069,0.082791,-0.041090,-0.085037,88.979310,-0.060665,0.030825,-0.002913


In [181]:
with open('final_19.pickle', 'wb') as handle:
    pickle.dump(df_final, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [188]:
df_final[0:10]

,url,Team,Date,Opponent,Spread,Win_Loss,Home,Away,GP,Wins,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,OeFG%,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,P_S,FG_S,FGA_S,FG%_S,3P_S,3PA_S,3P%_S,FT_S,FTA_S,FT%_S,TRB_S,AST_S,STL_S,BLK_S,TOV_S,PF_S,FTr_S,3PAr_S,TS%_S,TRB%_S,AST%_S,STL%_S,BLK%_S,Team_vs,Date_vs,Opponent_vs,Spread_vs,Win_Loss_vs,Home_vs,Away_vs,GP_vs,Wins_vs,Tm_vs,Opp_vs,FG_vs,FGA_vs,FG%_vs,3P_vs,3PA_vs,3P%_vs,FT_vs,FTA_vs,FT%_vs,ORB_vs,TRB_vs,AST_vs,STL_vs,BLK_vs,TOV_vs,PF_vs,ORtg_vs,DRtg_vs,Pace_vs,FTr_vs,3PAr_vs,TS%_vs,TRB%_vs,AST%_vs,STL%_vs,BLK%_vs,OeFG%_vs,OTOV%_vs,ORB%_vs,OFT/FGA_vs,DeFG%_vs,DTOV%_vs,DRB%_vs,DFT/FGA_vs,P_S_vs,FG_S_vs,FGA_S_vs,FG%_S_vs,3P_S_vs,3PA_S_vs,3P%_S_vs,FT_S_vs,FTA_S_vs,FT%_S_vs,TRB_S_vs,AST_S_vs,STL_S_vs,BLK_S_vs,TOV_S_vs,PF_S_vs,FTr_S_vs,3PAr_S_vs,TS%_S_vs,TRB%_S_vs,AST%_S_vs,STL%_S_vs,BLK%_S_vs
0,/cbb/boxscores/2018-11-13-22-california.html,California,2018-11-13,Hampton,14,1,1,0,1.0,0.000000,59.000000,76.000000,18.000000,51.000000,0.353000,2.000000,10.000000,0.200000,21.000000,32.000000,0.656000,7.000000,28.000000,6.000000,6.000000,1.000000,16.000000,17.000000,77.600000,100.000000,76.000000,0.627000,0.196000,0.446000,43.800000,33.300000,7.900000,3.200000,0.373000,19.500000,18.900000,0.412000,0.567000,22.900000,77.800000,0.327000,-17.000000,-7.000000,-1.000000,-0.128000,-7.000000,-11.000000,-0.229000,4.000000,7.000000,-0.024000,-8.000000,-8.000000,-1.000000,-3.000000,-3.000000,-12.000000,0.300077,-0.284444,-0.157175,86.600000,-0.105263,-0.013158,-0.039474,Hampton,2018-11-13,California,-14,0,0,1,1.0,0.000000,57.000000,69.000000,18.000000,62.000000,0.290000,3.000000,17.000000,0.176000,18.000000,24.000000,0.750000,12.000000,41.000000,7.000000,2.000000,3.000000,17.000000,26.000000,71.300000,86.300000,79.600000,0.387000,0.274000,0.388000,52.600000,38.900000,2.500000,8.800000,0.315000,18.800000,26.700000,0.290000,0.472000,18.900000,87.900000,0.333000,-12.000000,-5.000000,8.000000,-0.136000,-2.000000,-3.000000,-0.074000,0.000000,-7.000000,0.169000,4.000000,-7.000000,-6.000000,-1.000000,1.000000,1.000000,0.053667,0.074879,-0.122053,104.200000,-0.087940,-0.075377,-0.012563
1,/cbb/boxscores/2018-11-26-21-california.html,California,2018-11-26,Santa Clara,12,1,1,0,4.0,0.250000,69.250000,75.000000,23.500000,50.250000,0.467250,6.500000,14.250000,0.437000,15.750000,23.250000,0.677500,4.250000,27.000000,10.500000,5.000000,3.500000,13.000000,17.000000,97.600000,105.850000,71.275000,0.460750,0.284250,0.567000,46.175000,44.225000,7.025000,8.150000,0.532250,17.500000,14.275000,0.310750,0.536000,15.775000,76.550000,0.246250,-5.750000,-3.250000,-8.000000,-0.003250,-0.750000,-6.750000,0.091500,1.500000,4.250000,-0.044000,-4.500000,-4.000000,-1.500000,0.500000,0.500000,-4.000000,0.214716,-0.191778,-0.007820,91.350000,-0.059476,-0.022072,0.005725,Santa Clara,2018-11-26,California,-12,0,0,1,5.0,0.200000,63.600000,73.400000,23.400000,57.800000,0.402200,8.200000,24.800000,0.328200,8.600000,15.400000,0.564800,7.600000,29.800000,12.200000,5.200000,4.200000,13.400000,22.800000,88.480000,101.680000,71.800000,0.270200,0.428200,0.488400,46.340000,52.760000,7.180000,10.220000,0.473200,16.820000,22.040000,0.151600,0.485000,14.300000,70.980000,0.297600,-9.800000,-1.400000,-0.800000,-0.024800,1.400000,6.000000,-0.026000,-8.400000,-9.600000,-0.119000,-4.600000,-1.400000,-0.400000,1.000000,1.800000,5.000000,-0.027312,0.323610,-0.041497,91.680000,-0.019679,-0.006209,0.015796
2,/cbb/boxscores/2018-12-08-22-california.html,California,2018-12-08,San Diego State,6,1,1,0,7.0,0.285714,69.428571,75.571429,24.000000,51.714286,0.463143,6.142857,15.285714,0.395571,15.285714,21.142857,0.742429,5.000000,25.428571,10.000000,5.714286,3.285714,11.571429,17.428571,100.757143,110.171429,69.171429,0.412286,0.298143,0.562714,44.485714,42.528571,8.185714,9.114286,0.523143,15.771429,16.657143,0.295857,0.565714,17.271429,72.728571,0.249286,-6.142857,-2.857143,-4.000000,-0.027143

In [ ]:
df_final.